In [1]:
import json
import glob
import numpy as np
import pandas as pd
import os
from collections import defaultdict

In [2]:
from stats import get_dataframe

In [3]:
df = get_dataframe()

In [30]:
from scipy.special import betaln
def confidence_b_gr_a(pos_a, neg_a, pos_b, neg_b):
    alpha_a = pos_a + 1
    beta_a = neg_a + 1
    alpha_b = pos_b + 1
    beta_b = neg_b + 1
    
    total = 0.0
    for i in range(alpha_b):
        total += np.exp(
            betaln(alpha_a + i, beta_b + beta_a)
            - np.log(beta_b + i)
            - betaln(1 + i, beta_b)
            - betaln(alpha_a, beta_a)
        )
    return total

In [31]:
confidence_b_gr_a(57, 44, 23, 10)

0.9061748549294081

In [40]:
def compare_humans_and_bots(df, fields):
    human_stats = df[df.is_bot == False].groupby(fields + ['win']).count().game
    bot_stats = df[df.is_bot == True].groupby(fields + ['win']).count().game
    
    results = []
    for idx in human_stats.index:
        if not idx[-1]:
            continue
        idx = idx[:-1]
        human_win = human_stats.loc[idx + (True,)]
        human_loss = human_stats.loc[idx + (False,)]
        bot_win = bot_stats.loc[idx + (True,)]
        bot_loss = bot_stats.loc[idx + (False,)]
        new_dict = {
            'bot_winrate': bot_win / float(bot_win + bot_loss),
            'bot_n': bot_win + bot_loss,
            'human_winrate': human_win / float(human_win + human_loss),
            'human_n': human_win + human_loss,
            'bot_better_confidence': confidence_b_gr_a(human_win, human_loss, bot_win, bot_loss)
        }
        for k, v in zip(fields, idx):
            new_dict[k] = v
        results.append(new_dict)
    
    result = pd.DataFrame(results, columns=fields + ['bot_winrate', 'human_winrate', 'bot_better_confidence', 'bot_n', 'human_n'])
    return result.groupby(fields).mean()

compare_humans_and_bots(df, ['num_bots', 'is_resistance'])

bot_winrate  human_winrate  bot_better_confidence  \
num_bots is_resistance                                                      
1        False             0.696970       0.564356               0.906175   
         True              0.500000       0.383234               0.897890   
2        False             0.533742       0.545098               0.409464   
         True              0.454902       0.462366               0.427696   
3        False             0.675732       0.628125               0.917631   
         True              0.356050       0.324268               0.870641   
4        False             0.691037       0.563107               1.000000   
         True              0.352494       0.264507               0.999998   

                        bot_n  human_n  
num_bots is_resistance                  
1        False             33      101  
         True              34      167  
2        False            163      255  
         True             255      372  
3        False            478      320  
         True             719      478  
4        False           1997      515  
         True            3027      741

In [38]:
compare_humans_and_bots(df[df.role == 'Assassin'], ['num_bots'])

,bot_winrate,human_winrate,bot_better_confidence,bot_n,human_n
num_bots,,,,,
1,0.619048,0.577778,0.613533,21,45
2,0.590361,0.512000,0.864992,83,125
3,0.680851,0.625806,0.869695,235,155
4,0.697183,0.550201,0.999993,994,249


In [39]:
compare_humans_and_bots(df[df.role == 'Merlin'], ['num_bots'])

,bot_winrate,human_winrate,bot_better_confidence,bot_n,human_n
num_bots,,,,,
1,0.500000,0.392857,0.743813,10,56
2,0.454545,0.466102,0.435811,88,118
3,0.382609,0.282209,0.980524,230,163
4,0.341390,0.292000,0.929381,993,250
